# Archiving workspace for dealing with messy backup files 

This ipython notebook contains several functions for exploring backup files, investigating their start and end dates, cleaning them up, and outputting them to a more succinct .dat files. Run all of the function definitions before trying to process anything new. I am working on adding funcitonality that would set the cap length of a file to something approachable and workable like 200MB.

Modified by: Julia Signell

Date created: 2014-11-10

Date modified: 2015-04-01

In [1]:
from __future__ import print_function
import re
import os
import subprocess
import pandas as pd
import sys
import datetime as dt
import numpy as np
import shutil

In [2]:
FILEDIR = 'E:/TowerDataArchive/'
input_dir = FILEDIR

In [3]:
def search_and_replace(f,search,replace):
    #checks a file (f) for the search term and replaces it inplace 
    import fileinput

    for line in fileinput.input(f, inplace=True):
        print(line.replace(search,replace),end = '')
        
for f in os.listdir(input_dir):
    if 'Table1' in f:
        print(f)
        search_and_replace(input_dir+f,'Gass','Grass')
        search_and_replace(input_dir+f,'open','Open')

CR1000_SN5709_Table1_2012_05_21.dat
CR1000_SN5709_Table1_2010_04_23.dat
CR1000_SN5709_Table1_2010_07_18.dat
CR3000_SN3557_Table1_2012_10_27.dat
CR3000_SN3557_Table1_2010_02_21.dat
CR3000_SN3557_Table1_2010_04_19.dat
CR3000_SN3557_Table1_2011_07_07.dat
CR3000_SN3557_Table1_2012_10_25.dat
CR3000_SN3557_Table1_2010_07_19.dat
CR3000_SN3557_Table1_2010_04_13.dat
CR3000_SN3557_Table1_2010_02_08.dat
CR3000_SN3557_Table1_2011_12_31.dat
CR3000_SN3557_Table1_2012_02_28.dat
CR3000_SN3557_Table1_2010_07_20.dat
CR3000_SN3557_Table1_2010_03_09.dat
CR3000_SN5355_Table1_2014_09_02.dat
CR3000_SN5355_Table1_2012_10_29.dat


In [4]:
for f in os.listdir(input_dir):
    if 'Table1' in f:
        print(f)
        inFile = open(input_dir+f, 'rU')
        lines = inFile.readlines()
        if 'Gass' in lines[1]: print('still there')

CR1000_SN5709_Table1_2012_05_21.dat
CR1000_SN5709_Table1_2010_04_23.dat
CR1000_SN5709_Table1_2010_07_18.dat
CR3000_SN3557_Table1_2012_10_27.dat
CR3000_SN3557_Table1_2010_02_21.dat
CR3000_SN3557_Table1_2010_04_19.dat
CR3000_SN3557_Table1_2011_07_07.dat
CR3000_SN3557_Table1_2012_10_25.dat
CR3000_SN3557_Table1_2010_07_19.dat
CR3000_SN3557_Table1_2010_04_13.dat
CR3000_SN3557_Table1_2010_02_08.dat
CR3000_SN3557_Table1_2011_12_31.dat
CR3000_SN3557_Table1_2012_02_28.dat
CR3000_SN3557_Table1_2010_07_20.dat
CR3000_SN3557_Table1_2010_03_09.dat
CR3000_SN5355_Table1_2014_09_02.dat
CR3000_SN5355_Table1_2012_10_29.dat


In [4]:
def createDF(fileDir, df_dataName, data):
    #make a dataframe of file names with some file meta data in the columns
    files =[]
    for fileName in os.listdir(fileDir+data+'/'):
        if fileName not in open(fileDir+'processed2netCDF.txt','r').read():
        #if data in fileName:
            files.append(fileName)
            df_dataName=pd.DataFrame(index=[files], columns=['start','end','length','version'])
    return df_dataName, files

In [5]:
dateRE = re.compile(r'(\d{4})-(\d{2})-(\d{2})')
dateStringRE = re.compile(r'(\d{4})-(\d{2})-(\d{2}) (\d{2}):(\d{2}):(\d{2})\.?(\d)?')
def parseCampbellDate(dateString):
    date = dateStringRE.match(dateString)
    try:
        return ([int(date.group(j)) for j in range(1,7)]+[int(date.group(7))*100000])
    except:
        return ([int(date.group(j)) for j in range(1,7)])

def numericDate(dateString):
    return datetime.datetime(*parseCampbellDate(dateString))

In [6]:
# Open a file and figure out when it starts and ends
def daterange(fileDir,fileName,df_dataName): 
    print fileName
    inFile = open(fileDir+fileName,'rU')
    lines = inFile.readlines()
    try:
        start = lines[4][1:20]
        start = numericDate(start)
    except: start = nan      
    try: 
        end = lines[-1][1:20]
        end = numericDate(end)
    except: end = nan
    df_dataName['start'][fileName]=start
    df_dataName['end'][fileName]=end
    try: length = end-start
    except: length = nan
    df_dataName['length'][fileName]=length
    df_dataName['version'][fileName]=lines[0].partition('CPU:')[2].partition('"')[0]
    return df_dataName

In [7]:
def process(fileDir,data,df_dataName):
    df_dataName, files = createDF(fileDir, df_dataName, data)
    for fileName in files:
        try:df_dataName = daterange(fileDir, fileName,df_dataName)
        except: continue
    df_dataName = df_dataName.sort(columns = ['version','start','length'], ascending = [True,True,False])
    df_dataName.to_csv(fileDir+ '%s/'%data + '%s_summary.csv' %data)
    
    #df_dataName[['start']] = df_dataName[['start']].drop_duplicates()
    df_dataName = df_dataName.dropna(how ='any')
    goagain =True
    while goagain ==True:
        goagain = False
        i=0
        while i in(range(len(df_dataName.index)-1)):
            j = df_dataName.iloc[i,:]
            k = df_dataName.iloc[i+1,:]
            if k['version'] == j['version'] and k['start']<=j['end'] and k['end']<=j['end']:
                df_dataName = df_dataName.drop(k.name)
                goagain = True
            i+=1
        print goagain
    print 'done!'
    return df_dataName

In [8]:
def mergeFiles(fileDir,data,df_dataName):
    fileNames = df.index
    day = str(df_dataName['start'][fileNames[0]]).replace('-','_').partition(' ')[0]
    number = fileNames[0].split(data)[0]
    outFile = open(fileDir.partition('towerraw/')[0]+number+data+'_'+day+'.dat','w')
    for i in range(len(fileNames)-1):
        oldFile = open(fileDir+data+'/'+fileNames[i],'rU')
        oldlines = oldFile.readlines()
        newFile = open(fileDir+data+'/'+fileNames[i+1],'rU')
        newlines = newFile.readlines()
        if oldlines[0:4] != newlines[0:4]:
            outFile.close()
            day = str(df_dataName['start'][fileNames[i+1]]).replace('-','_').partition(' ')[0]
            number = fileNames[i+1].split(data)[0]
            outFile = open(fileDir.partition('towerraw/')[0]+number+data+'_'+day+'.dat','w')
            for line in newlines:
                print >>outFile, line,
        elif oldlines[0:4] == newlines[0:4]:       
            for line in newlines[4:]:
                if line not in oldlines:
                    print >>outFile, line,
    print 'done merging!'
    outFile.close

In [82]:
'''def mergeFiles(fileDir,data,df_dataName):
    versions = list(df_dataName['version'].unique())
    k = 0
    number=list()
    for version in versions:
        outFile = open(fileDir+data+'_'+version+'.dat','w')
        i = 0
        for fileName in df_dataName.index:
            if df_dataName['version'][fileName] == version:
                print 'merging' + fileName
                inFile = open(fileDir+data+'/'+fileName,'rU')
                lines = inFile.readlines()
                if i ==0:
                    number.append(fileName.split(data)[0])
                    print >>outFile, lines[0],lines[1],lines[2],lines[3],
                    i+=1
                if not(lines[0][0:6] == '"TOA5"'): raise Exception ('file %s does not have a header'%fileName)
                for i in range(4,len(lines)):
                    print >>outFile, lines[i],
        outFile.close()
        lines_seen = set() # holds lines already seen
        outfile = open(fileDir.partition('towerraw/')[0]+number[k]+data+'_'+version+'.dat', "w")
        k+=1
        for line in open(fileDir+data+'_'+version+'.dat', "r"):
            if line not in lines_seen: # not a duplicate
                outfile.write(line)
                lines_seen.add(line)
        os.remove(fileDir+data+'_'+version+'.dat')
        outfile.close()
    print 'done!''''

So this is where you need to start paying attention. If you have already run everything above this, then you should be ready to actually process files. This is where you need to input the name of the data value that you are interested in.

In [157]:
fileDir = 'C:/Users/Julia/Dropbox (PE)/KenyaLab/Data/Tower/TowerDataArchive/towerraw/'
df = pd.DataFrame
data = 'licor'
for fileName in os.listdir(fileDir):
    if data in fileName:
        try:
            shutil.copy(fileDir+fileName,fileDir+ '%s/'%data+fileName)
            os.remove(fileDir+fileName)
        except: continue
df = process(fileDir,data,df)
mergeFiles(fileDir,data,df)

CR1000_SN13341_licor6262.dat
CR1000_SN13341_licor6262.dat.1.backup
CR1000_SN13341_licor6262.dat.2.backup
CR1000_SN13341_licor6262.dat.3.backup
CR1000_SN13341_licor6262.dat.backup
CR1000_SN13341_licor6262_2012_10_29.dat
CR1000_SN13341_licor6262_AVG_2013_01_08.dat
CR800_SN4735_licor6262.dat
False

MemoryError: 


done!


In [154]:
fileDir = "C:/Users/Julia/Dropbox (PE)/KenyaLab/Data/Tower/TowerDataArchive/"
data = 'flux'
for fileName in os.listdir(fileDir):
    if data in fileName:
        print fileName
        df1 = pd.read_csv(fileDir+fileName,header=[1,2,3], parse_dates = 0,index_col = 0)
        for i in range(0,len(df1.index)-1):
            if df1.index[i+1]-df1.index[i] > datetime.timedelta(0, 1800):
                print df1.index[i],'missing until',df1.index[i+1]

CR1000_SN7276_flux_2010_02_04.dat
CR1000_SN7276_flux_2010_02_05.dat
2010-02-09 11:00:00 missing until 2010-02-09 12:00:00
2010-02-19 10:00:00 missing until 2010-02-19 11:00:00
2010-02-19 11:00:00 missing until 2010-02-19 12:30:00
2010-02-20 09:30:00 missing until 2010-02-20 10:30:00
2010-03-06 01:00:00 missing until 2010-03-06 09:30:00
CR1000_SN7276_flux_2010_04_01.dat
2010-04-09 12:30:00 missing until 2010-04-09 13:30:00
2010-04-11 04:30:00 missing until 2010-04-11 05:30:00
2010-04-11 12:00:00 missing until 2010-04-11 13:00:00
2010-04-11 14:00:00 missing until 2010-04-11 15:00:00
2010-04-11 19:30:00 missing until 2010-04-11 20:30:00
2010-04-12 04:30:00 missing until 2010-04-12 05:30:00
2010-04-12 19:00:00 missing until 2010-04-12 20:00:00
2010-04-12 22:30:00 missing until 2010-04-12 23:30:00
2010-04-13 05:00:00 missing until 2010-04-13 06:00:00
2010-04-13 10:00:00 missing until 2010-04-13 11:00:00
CR1000_SN7276_flux_2010_04_26.dat
CR1000_SN7276_flux_2010_04_27.dat
CR1000_SN7276_flux_2

In [64]:
df.head()

TIMESTAMP,RECORD,Year,Month,DOM,Hour,Minute,Second,uSecond,WeekDay,Day_of_Year,...,Solar_Wm2_Avg,Solar_MJ_Tot,Solar_Wm2_1_Avg,Solar_MJ_1_Tot,e_hmp_Avg,e_hmp_Std,t_hmp_Avg,t_hmp_Std,rh_hmp_Avg,rh_hmp_Std
TS,RN,Unnamed: 2_level_1,Unnamed: 3_level_1,Unnamed: 4_level_1,Unnamed: 5_level_1,Unnamed: 6_level_1,Unnamed: 7_level_1,Unnamed: 8_level_1,Unnamed: 9_level_1,Unnamed: 10_level_1,...,W/m2,MJ/m2,W/m2,MJ/m2,kPa,kPa,C,C,percent,percent
,Unnamed: 1_level_2,Smp,Smp,Smp,Smp,Smp,Smp,Smp,Smp,Smp,...,Avg,Tot,Avg,Tot,Avg,Std,Avg,Std,Avg,Std
2011-07-07 14:00:00,0,2011,7,7,14,0,0,0,5,188,...,NAN,NAN,NAN,NAN,0,0,0,0,0,0
2011-07-07 14:10:00,1,2011,7,7,14,10,0,0,5,188,...,NAN,NAN,NAN,NAN,0,0,0,0,0,0
2011-07-07 14:20:00,2,2011,7,7,14,20,0,0,5,188,...,NAN,NAN,NAN,NAN,0,0,0,0,0,0
2011-07-07 14:30:00,3,2011,7,7,14,30,0,0,5,188,...,NAN,NAN,NAN,NAN,0,0,0,0,0,0
2011-07-07 14:40:00,4,2011,7,7,14,40,0,0,5,188,...,NAN,NAN,NAN,NAN,0,0,0,0,0,0


In [ ]:
fileDir = 'C:/Users/Julia/Dropbox (PE)/KenyaLab/Data/Tower/TowerDataArchive/towerraw/'
df = pd.DataFrame
data = 'upper'
for fileName in os.listdir(fileDir):
    if data in fileName:
        try:
            shutil.copy(fileDir+fileName,fileDir+ '%s/'%data+fileName)
            os.remove(fileDir+fileName)
        except: continue
df = process(fileDir,data,df)
mergeFiles(fileDir,data,df)

In [45]:
k = 0
for root, dirs, files in os.walk('E:/'):
    for i in files:
        if 'upper' in i:
            print root+'/'+i
            k+=1
print k

66


In [15]:
DATALOC = "E:/TowerDataArchive/"
fileDir = DATALOC
df_dataName = pd.DataFrame
df_dataName, files = createDF(fileDir, df_dataName,'')
for fileName in files:
    df_dataName = daterange(fileDir, fileName,df_dataName)

archiving.ipynb
CR1000_SN13341_licor_2012_09_16.dat
CR3000_SN4709_flux_2010_04_23.dat
CR3000_SN4709_WVIA_2012_10_23.dat
CR3000_SN4709_WVIA_2014_05_27.dat
desktop.ini
processed2netCDF.txt
test

IOError: [Errno 13] Permission denied: 'C:/Users/Julia/Dropbox (PE)/KenyaLab/Data/Tower/TowerDataArchive/test'